In [1]:
import sys
import yaml
import numpy as np
import pandas as pd
import pyntcloud
import importlib as imp

num ='000'
scan = np.fromfile('../dataset/sequences/00/velodyne/000' + num + '.bin', dtype=np.float32)
scan = scan.reshape((-1, 4))

label = np.fromfile('../dataset/sequences/00/labels/000' + num + '.label', dtype=np.uint32)
label = label.reshape((-1))

In [2]:
with open('../config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
obstacle_dct = config['obstacles']

In [4]:
# seg id -> seg name
obstacle_dct

{10: 'car',
 11: 'bicycle',
 13: 'bus',
 15: 'motorcycle',
 16: 'on-rails',
 18: 'truck',
 20: 'other-vehicle',
 30: 'person',
 31: 'bicyclist',
 32: 'motorcyclist',
 252: 'moving-car',
 253: 'moving-bicyclist',
 254: 'moving-person',
 255: 'moving-motorcyclist',
 256: 'moving-on-rails',
 257: 'moving-bus',
 258: 'moving-truck',
 259: 'moving-other-vehicle'}

In [5]:
leaning_map_obstacles = config['learning_map']

In [6]:
# ground truth id -> learning id
# leaning_map_obstacles

In [7]:
mapped_obstacles_lst = list(set([leaning_map_obstacles[id] for id in obstacle_dct.keys()]))
mapped_obstacles_lst

[1, 2, 3, 4, 5, 6, 7, 8]

In [40]:
import sys
sys.path.append("..")
from model.tasks.semantic import infer2
infer2 = imp.reload(infer2)

### Load Model

In [43]:
user = infer2.get_user("squeezeseg-crf")

----------
INTERFACE:
model squeezeseg-crf
----------

Opening arch config file from squeezeseg-crf
Opening data config file from squeezeseg-crf
model folder exists! Using model from squeezeseg-crf
parser started
Using SqueezeNet Backbone
Depth of backbone input =  5
Original OS:  16
New OS:  16
Strides:  [2, 2, 2, 2]
Decoder original OS:  16
Decoder new OS:  16
Decoder strides:  [2, 2, 2, 2]
Using CRF!
Total number of parameters:  915965
Total number of parameters requires_grad:  915960
Param encoder  724032
Param decoder  179968
Param head  11540
Param CRF  425
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Successfully loaded model CRF weights
Infering in device:  cpu


In [44]:
scan = np.fromfile('../dataset/sequences/00/velodyne/000000.bin', dtype=np.float32)
scan = scan.reshape((-1, 4))
%timeit predicted_labels = user.infer(scan)

1.95 s ± 739 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
predicted_labels = user.infer(scan)

In [46]:
predicted_labels.shape

(124668,)

### Visualize segmentated point cloud

In [13]:
cloud = pd.DataFrame(scan[:, :3], columns=['x', 'y', 'z'])
cloud['seg_id'] = predicted_labels
pcloud = pyntcloud.PyntCloud(cloud)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-1.4028828144073486, 52.310731291770935, 3.0…

![lol](img/darknet12_scan_000000_seq_00.png)

In [79]:
pcloud.points.seg_id.unique()

array([16, 15, 13, 19, 18, 14,  1,  8,  9, 10, 17, 11,  5,  7])

### Visualize only obstacles

In [ ]:
cloud_selected  = cloud[cloud.seg_id.isin(mapped_obstacles_lst)]
pcloud = pyntcloud.PyntCloud(cloud_selected)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

![lol](img/darknet12_scan_000000_seq_00_obstacles.png)

### Load Model

In [14]:
user = get_user( "squeezeseg")

----------
INTERFACE:
model squeezeseg
----------

Opening arch config file from squeezeseg
Opening data config file from squeezeseg
model folder exists! Using model from squeezeseg
parser started
Using SqueezeNet Backbone
Depth of backbone input =  5
Original OS:  16
New OS:  16
Strides:  [2, 2, 2, 2]
Decoder original OS:  16
Decoder new OS:  16
Decoder strides:  [2, 2, 2, 2]
Total number of parameters:  915540
Total number of parameters requires_grad:  915540
Param encoder  724032
Param decoder  179968
Param head  11540
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Infering in device:  cpu


In [20]:
%timeit predicted_labels = user.infer(scan)

273 ms ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
predicted_labels = user.infer(scan)
predicted_labels.shape

(124605,)

### Visualize segmentated point cloud

In [ ]:
cloud = pd.DataFrame(scan[:, :3], columns=['x', 'y', 'z'])
cloud['seg_id'] = predicted_labels
pcloud = pyntcloud.PyntCloud(cloud)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

![lol](img/squeezeseg_scan_000000_seq_00.png)

In [25]:
pcloud.points.seg_id.unique()

array([18, 16, 19, 13, 15,  4,  5, 12,  1,  6,  8, 14, 17,  7, 10,  9, 11,
        3,  2])

### Visualize only obstacles

In [ ]:
cloud_selected  = cloud[cloud.seg_id.isin(mapped_obstacles_lst)]
pcloud = pyntcloud.PyntCloud(cloud_selected)
pcloud.plot(use_as_color='seg_id', cmap="cool", backend="pythreejs")

![lol](img/squeezeseg_scan_000000_seq_00_obstacles.png)